In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.text import Tokenizer
import gc                         
gc.enable()
from sklearn.linear_model import LogisticRegression #логистическая регрессия
from sklearn.metrics import accuracy_score

In [2]:
#загружаем данные из файла "transactions.csv"
df_transactions = pd.read_csv('/content/drive/MyDrive/transactions.csv', sep=',')

In [3]:
#формируем датафрейм с максимальным количеством заказов для каждого пользователя
df_user = df_transactions.groupby('user_id')[['order_number']].max()
df_user.columns = ['user_total_orders']
df_user = df_user.reset_index()
df_user.head()

,user_id,user_total_orders
0,1,10
1,2,14
2,3,11
3,7,20
4,13,12


In [4]:
#формируем вспомогательный датафрейм с деталями заказов
df_orders = df_transactions[['user_id', 'order_id', 'product_id', 'add_to_cart_order', 'reordered']]
df_orders.head()

,user_id,order_id,product_id,add_to_cart_order,reordered
0,1,2539329,196,1.0,0.0
1,1,2539329,14084,2.0,0.0
2,1,2539329,12427,3.0,0.0
3,1,2539329,26088,4.0,0.0
4,1,2539329,26405,5.0,0.0


In [5]:
#вычисляем среднее по столбцу 'reordered' для каждого пользователя и формируем датафрейм
df_user_reorder = df_transactions.groupby('user_id')['reordered'].mean().to_frame('user_reordered_ratio').reset_index()
df_user_reorder.head()

,user_id,user_reordered_ratio
0,1,0.694915
1,2,0.476923
2,3,0.597561
3,7,0.669903
4,13,0.641975


In [6]:
#вычисляем среднее значение позиции товара в чеке по каждой паре "пользователь/товар", затем сортируем по возрастанию
df_add_to_cart_order = df_transactions.groupby(['user_id', 'product_id'])['add_to_cart_order'].mean().to_frame('user_add_to_cart_order_ratio').reset_index()
df_add_to_cart_order = df_add_to_cart_order.sort_values(by=['user_id', 'user_add_to_cart_order_ratio'])
df_add_to_cart_order.head()

,user_id,product_id,user_add_to_cart_order_ratio
0,1,196,1.4
6,1,14084,2.0
14,1,39657,3.0
16,1,46149,3.0
3,1,12427,3.3


In [7]:
#объединяем df_transactions и df_user
df_transactions = pd.merge(df_transactions, df_user, on=['user_id'], how='left')

In [8]:
#добавляем столбец 'diff', представляющий собой разницу между общим кол-вом заказов и текущим номером заказа
df_transactions['diff'] = df_transactions['user_total_orders'] - df_transactions['order_number']
df_transactions.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,user_total_orders,diff
0,2539329,1,1,2,8,NaN,196,1.0,0.0,10,9
1,2539329,1,1,2,8,NaN,14084,2.0,0.0,10,9
2,2539329,1,1,2,8,NaN,12427,3.0,0.0,10,9
3,2539329,1,1,2,8,NaN,26088,4.0,0.0,10,9
4,2539329,1,1,2,8,NaN,26405,5.0,0.0,10,9
5,2398795,1,2,3,7,15.0,196,1.0,1.0,10,8
6,2398795,1,2,3,7,15.0,10258,2.0,0.0,10,8
7,2398795,1,2,3,7,15.0,12427,3.0,1.0,10,8
8,2398795,1,2,3,7,15.0,13176,4.0,0.0,10,8
9,2398795,1,2,3,7,15.0,26088,5.0,1.0,10,8


In [9]:
#объединяем df_user_reorder и df_user
df_user = df_user.merge(df_user_reorder, on='user_id', how='left')
df_user.head()

,user_id,user_total_orders,user_reordered_ratio
0,1,10,0.694915
1,2,14,0.476923
2,3,11,0.597561
3,7,20,0.669903
4,13,12,0.641975


In [10]:
del df_user_reorder
gc.collect()

45

In [11]:
#находим общее количество приобретенных товаров по каждому product_id	и формируем датафрейм
df_purchases = df_transactions.groupby('product_id')['order_id'].count().to_frame('product_total_purchases').reset_index()
df_purchases.head()

,product_id,product_total_purchases
0,1,1549
1,2,80
2,3,242
3,4,215
4,5,11


In [12]:
#вычисляем среднее по столбцу 'reordered' для каждого товара и формируем датафрейм
df_product_reorder = df_transactions.groupby('product_id')['reordered'].mean().to_frame('product_reorder_ratio').reset_index()
df_product_reorder.head()

,product_id,product_reorder_ratio
0,1,0.678502
1,2,0.137500
2,3,0.789256
3,4,0.488372
4,5,0.727273


In [13]:
#объединяем df_purchases и df_product_reorder
df_purchases = df_purchases.merge(df_product_reorder, on='product_id', how='left')
df_purchases.head()

,product_id,product_total_purchases,product_reorder_ratio
0,1,1549,0.678502
1,2,80,0.137500
2,3,242,0.789256
3,4,215,0.488372
4,5,11,0.727273


In [14]:
del df_product_reorder
gc.collect()

36

In [15]:
#удаляем Nan из столбца 'product_reorder_ratio'
df_purchases['product_reorder_ratio'] = df_purchases['product_reorder_ratio'].fillna(value=0)
df_purchases.head()

,product_id,product_total_purchases,product_reorder_ratio
0,1,1549,0.678502
1,2,80,0.137500
2,3,242,0.789256
3,4,215,0.488372
4,5,11,0.727273


In [17]:
#находим общее количество товаров по каждой паре "пользователь/товар", сортируем столбец по убыванию 'user_product_total_purchases'
df_user_product_purchases = df_transactions.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('user_product_total_purchases').reset_index()
df_user_product_purchases = df_user_product_purchases.sort_values(by=['user_id', 'user_product_total_purchases'], ascending = [True, False])
df_user_product_purchases.head()

,user_id,product_id,user_product_total_purchases
0,1,196,10
3,1,12427,10
1,1,10258,9
8,1,25133,8
4,1,13032,3


In [18]:
df_total_orders = df_transactions.groupby('user_id')['order_number'].max().to_frame('total_orders')
df_total_orders.head()

,total_orders
user_id,
1,10
2,14
3,11
7,20
13,12


In [19]:
#формируем датафрейм с номерами заказов, в которых впервые встречается каждая пара "пользователь/товар"
df_first_order = df_transactions.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number').reset_index()
df_first_order.head()

,user_id,product_id,first_order_number
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [20]:
#формируем датафрейм как объединение df_total_orders и df_first_order
df_span = pd.merge(df_total_orders, df_first_order, on='user_id', how='right')
df_span.head()

,user_id,total_orders,product_id,first_order_number
0,1,10,196,1
1,1,10,10258,2
2,1,10,10326,5
3,1,10,12427,1
4,1,10,13032,2


In [21]:
#добавляем столбец 'order_range_denominator', представляющий разность между общим количеством заказов и первым заказом
#по каждой паре "пользователь/товар", сортируем 'order_range_denominator' по убыванию
df_span['order_range_denominator'] = df_span['total_orders'] - df_span['first_order_number'] + 1
df_span = df_span.sort_values(by=['user_id', 'order_range_denominator'], ascending = [True, False])
df_span.head()

,user_id,total_orders,product_id,first_order_number,order_range_denominator
0,1,10,196,1,10
3,1,10,12427,1,10
6,1,10,14084,1,10
9,1,10,26088,1,10
10,1,10,26405,1,10


In [22]:
#объединяем df_user_product_purchases и df_span
df_user_product_ratio = pd.merge(df_user_product_purchases, df_span, on=['user_id', 'product_id'], how='left')
df_user_product_ratio.head()

,user_id,product_id,user_product_total_purchases,total_orders,first_order_number,order_range_denominator
0,1,196,10,10,1,10
1,1,12427,10,10,1,10
2,1,10258,9,10,2,9
3,1,25133,8,10,3,8
4,1,13032,3,10,2,9


In [23]:
#добавляем столбец 'user_product_reorder_ratio', представляющий собой отношение 
#общего кол-ва покупок к деноминатору заказа по каждой паре "пользователь/товар"
#сортируем 'user_product_reorder_ratio' по убыванию
df_user_product_ratio['user_product_reorder_ratio'] = df_user_product_ratio['user_product_total_purchases'] / df_user_product_ratio['order_range_denominator']
df_user_product_ratio = df_user_product_ratio.sort_values(by=['user_id', 'user_product_reorder_ratio'], ascending=[True, False])
df_user_product_ratio.head()

,user_id,product_id,user_product_total_purchases,total_orders,first_order_number,order_range_denominator,user_product_reorder_ratio
0,1,196,10,10,1,10,1.0
1,1,12427,10,10,1,10,1.0
2,1,10258,9,10,2,9,1.0
3,1,25133,8,10,3,8,1.0
5,1,46149,3,10,8,3,1.0


In [24]:
del df_user_product_ratio['user_product_total_purchases']
del df_user_product_ratio['total_orders']
del df_user_product_ratio['first_order_number']
del df_user_product_ratio['order_range_denominator']
gc.collect()
df_user_product_ratio.head()

,user_id,product_id,user_product_reorder_ratio
0,1,196,1.0
1,1,12427,1.0
2,1,10258,1.0
3,1,25133,1.0
5,1,46149,1.0


In [25]:
del df_span
del df_first_order
gc.collect()

12

In [26]:
#объединяем df_user_product_ratio и df_user_product_purchases
#сортируем 'user_product_reorder_ratio' по убыванию
df_user_product_purchases = df_user_product_purchases.merge(df_user_product_ratio, on=['user_id', 'product_id'], how='left')
df_user_product_purchases = df_user_product_purchases.sort_values(by=['user_id', 'user_product_reorder_ratio'], ascending=[True, False])
df_user_product_purchases.head()

,user_id,product_id,user_product_total_purchases,user_product_reorder_ratio
0,1,196,10,1.0
1,1,12427,10,1.0
2,1,10258,9,1.0
3,1,25133,8,1.0
5,1,46149,3,1.0


In [27]:
del df_user_product_ratio
gc.collect()

12

In [28]:
#добавляем столбец 'back_order_no', который представляет собой разницу между максимальным значением по столбцу 'order_number' 
#для каждого пользователя и текущим номером заказа
df_transactions['back_order_no'] = df_transactions.groupby('user_id')['order_number'].transform(max) - df_transactions['order_number'] + 1
df_transactions.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,user_total_orders,diff,back_order_no
0,2539329,1,1,2,8,NaN,196,1.0,0.0,10,9,10
1,2539329,1,1,2,8,NaN,14084,2.0,0.0,10,9,10
2,2539329,1,1,2,8,NaN,12427,3.0,0.0,10,9,10
3,2539329,1,1,2,8,NaN,26088,4.0,0.0,10,9,10
4,2539329,1,1,2,8,NaN,26405,5.0,0.0,10,9,10


In [29]:
#формируем датафрейм, состоящий только из последних пяти заказов каждого пользователя
df_transactions_5 = df_transactions.loc[df_transactions['back_order_no'] <= 5]
df_transactions_5.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,user_total_orders,diff,back_order_no
29,3367565,1,6,2,7,19.0,196,1.0,1.0,10,4,5
30,3367565,1,6,2,7,19.0,12427,2.0,1.0,10,4,5
31,3367565,1,6,2,7,19.0,10258,3.0,1.0,10,4,5
32,3367565,1,6,2,7,19.0,25133,4.0,1.0,10,4,5
33,550135,1,7,1,9,20.0,196,1.0,1.0,10,3,4


In [30]:
del df_transactions
gc.collect()

37

In [31]:
#создаем тренировочный и тестовый датасеты
#train представляет собой все заказы, за исключением последних двух, test - последние два заказа всех пользователей
train = df_transactions_5.loc[df_transactions_5['diff']>1]
test = df_transactions_5.loc[df_transactions_5['diff']<=1]

In [32]:
#формируем датафрейм с количеством заказов, в которых присутствовали определенные товары в последних пяти заказах
orders_5_train = train.groupby(['user_id','product_id'])[['order_id']].count()
orders_5_train.columns = ['last_5']
orders_5_train.head()

last_5
user_id product_id        
1       196              3
        10258            3
        12427            3
        13032            1
        25133            3

In [33]:
#объединяем orders_5_train с датафреймом, представляющим только те товары, которые перезаказывались
orders_6_train = df_transactions_5.groupby(['user_id','product_id'])[['reordered']].max()
orders_5_train = orders_5_train.merge(orders_6_train, on=['user_id', 'product_id'], how='left')
orders_5_train.head()

last_5  reordered
user_id product_id                   
1       196              3        1.0
        10258            3        1.0
        12427            3        1.0
        13032            1        1.0
        25133            3        1.0

In [34]:
del orders_6_train
gc.collect()

12

In [35]:
#объединяем df_user_product_purchases и orders_5_train
df_user_product_purchases_train = df_user_product_purchases.merge(orders_5_train, on=['user_id', 'product_id'], how='inner')
df_user_product_purchases_train.head()

,user_id,product_id,user_product_total_purchases,user_product_reorder_ratio,last_5,reordered
0,1,196,10,1.0,3,1.0
1,1,12427,10,1.0,3,1.0
2,1,10258,9,1.0,3,1.0
3,1,25133,8,1.0,3,1.0
4,1,46149,3,1.0,1,1.0


In [36]:
#объединяем df_user_product_purchases_train и df_user
train = df_user_product_purchases_train.merge(df_user, on='user_id', how='inner')
train.head()

,user_id,product_id,user_product_total_purchases,user_product_reorder_ratio,last_5,reordered,user_total_orders,user_reordered_ratio
0,1,196,10,1.0,3,1.0,10,0.694915
1,1,12427,10,1.0,3,1.0,10,0.694915
2,1,10258,9,1.0,3,1.0,10,0.694915
3,1,25133,8,1.0,3,1.0,10,0.694915
4,1,46149,3,1.0,1,1.0,10,0.694915


In [37]:
#объединяем train и df_purchases
train = train.merge(df_purchases, on='product_id', how='inner')
train.head()

,user_id,product_id,user_product_total_purchases,user_product_reorder_ratio,last_5,reordered,user_total_orders,user_reordered_ratio,product_total_purchases,product_reorder_ratio
0,1,196,10,1.000000,3,1.0,10,0.694915,28819,0.82921
1,15,196,4,0.571429,2,1.0,21,0.814286,28819,0.82921
2,31,196,2,0.200000,1,1.0,19,0.367347,28819,0.82921
3,43,196,2,0.333333,1,1.0,11,0.401361,28819,0.82921
4,67,196,19,0.791667,2,1.0,24,0.716049,28819,0.82921


In [38]:
#формируем датафрейм с количеством заказов, в которых присутствовали определенные товары в последних пяти заказах
orders_5_test = test.groupby(['user_id','product_id'])[['order_id']].count()
orders_5_test.columns = ['last_5']
orders_5_test.head()

last_5
user_id product_id        
1       196              2
        10258            2
        12427            2
        13032            1
        25133            2

In [39]:
#объединяем orders_5_test с датафреймом, представляющим только те товары, которые перезаказывались
orders_6_test = df_transactions_5.groupby(['user_id','product_id'])[['reordered']].max()
orders_5_test = orders_5_test.merge(orders_6_test, on=['user_id', 'product_id'], how='left')
orders_5_test.head()

last_5  reordered
user_id product_id                   
1       196              2        1.0
        10258            2        1.0
        12427            2        1.0
        13032            1        1.0
        25133            2        1.0

In [40]:
del orders_6_test
gc.collect()

36

In [41]:
#объединяем df_user_product_purchases и orders_5_test
df_user_product_purchases_test = df_user_product_purchases.merge(orders_5_test, on=['user_id', 'product_id'], how='inner')
df_user_product_purchases_test.head()

,user_id,product_id,user_product_total_purchases,user_product_reorder_ratio,last_5,reordered
0,1,196,10,1.0,2,1.0
1,1,12427,10,1.0,2,1.0
2,1,10258,9,1.0,2,1.0
3,1,25133,8,1.0,2,1.0
4,1,46149,3,1.0,2,1.0


In [42]:
#объединяем df_user_product_purchases_test и df_user
test = df_user_product_purchases_test.merge(df_user, on='user_id', how='inner')
test = test.merge(df_purchases, on='product_id', how='inner')
test.head()

,user_id,product_id,user_product_total_purchases,user_product_reorder_ratio,last_5,reordered,user_total_orders,user_reordered_ratio,product_total_purchases,product_reorder_ratio
0,1,196,10,1.000000,2,1.0,10,0.694915,28819,0.82921
1,15,196,4,0.571429,1,1.0,21,0.814286,28819,0.82921
2,67,196,19,0.791667,1,1.0,24,0.716049,28819,0.82921
3,82,196,9,0.473684,2,1.0,19,0.793893,28819,0.82921
4,98,196,6,0.545455,2,1.0,13,0.437500,28819,0.82921


In [43]:
del df_user,
del df_purchases
del df_user_product_purchases
gc.collect()

65

In [44]:
test

,user_id,product_id,user_product_total_purchases,user_product_reorder_ratio,last_5,reordered,user_total_orders,user_reordered_ratio,product_total_purchases,product_reorder_ratio
0,1,196,10,1.000000,2,1.0,10,0.694915,28819,0.829210
1,15,196,4,0.571429,1,1.0,21,0.814286,28819,0.829210
2,67,196,19,0.791667,1,1.0,24,0.716049,28819,0.829210
3,82,196,9,0.473684,2,1.0,19,0.793893,28819,0.829210
4,98,196,6,0.545455,2,1.0,13,0.437500,28819,0.829210
...,...,...,...,...,...,...,...,...,...,...
1839777,206165,6537,1,0.500000,1,0.0,27,0.431127,8,0.250000
1839778,206165,35937,3,0.428571,1,1.0,27,0.431127,16,0.187500
1839779,206183,8133,1,0.500000,1,0.0,13,0.664865,11,0.181818
1839780,206201,33935,1,0.500000,1,0.0,31,0.563776,8,0.250000


In [47]:
y_train = train['reordered'].ravel()#target тренировочный
x_train = train.drop(['reordered'], axis=1)#признаки без target тренировочные
y_test = test['reordered'].ravel()#target тестовый
x_test = test.drop(['reordered'], axis=1)#признаки без target тестовые

In [48]:
logreg = LogisticRegression(C=0.9, random_state=17, max_iter=1000)#определяем алгоритм с коэффициентом регуляризации 0.9, максимальным кол-вом итераций 1000
logreg_model = logreg.fit(x_train, y_train)#обучаем алгоритм на тренировочных данных

In [49]:
logreg_model_prediction = logreg_model.predict(x_test)#предсказания модели на тестовых данных

In [50]:
print(f'accuracy_score на валидационных данных: {accuracy_score(y_test, logreg_model_prediction)}')#точность предсказаний

accuracy_score на валидационных данных: 0.9023248406604696


In [51]:
#функция предназначена для приведения списка к формату, который требуется в задаче
def clean_prediction(row):
    data = row.product_id
    data = str("".join(str(data))[1:-1].replace(',', ' '))
    return data

In [60]:
#функция предназначена для создания датафрейма из тестового набора данных и списка полученных предсказаний
def create_predicted_df(users, products, predictions, columns=['user_id', 'product_id', 'predictions']):
    predicted_df = pd.DataFrame(list(zip(users, products, predictions)), columns=columns)
    predicted_df['result'] = (predicted_df['product_id'] * predicted_df['predictions']).astype(int)
    return predicted_df

In [61]:
#создаем датафрейм с помощью функции "create_predicted_df"
df_total = create_predicted_df(x_test['user_id'], x_test['product_id'], logreg_model_prediction, columns=['user_id', 'product_id', 'predictions'])

In [62]:
df_total

,user_id,product_id,predictions,result
0,1,196,1.0,196
1,15,196,1.0,196
2,67,196,1.0,196
3,82,196,1.0,196
4,98,196,1.0,196
...,...,...,...,...
1839777,206165,6537,0.0,0
1839778,206165,35937,1.0,35937
1839779,206183,8133,0.0,0
1839780,206201,33935,0.0,0


In [71]:
#для каждого пользователя получаем список предсказаний из столбца 'result' с помощью "apply(list)"
rec = df_total.groupby('user_id')['result'].apply(list).reset_index(name='product_id')

In [72]:
rec

,user_id,product_id
0,1,"[196, 12427, 10258, 25133, 46149, 35951, 38928..."
1,2,"[7963, 20785, 22825, 24768, 27413, 0, 47209, 1..."
2,3,"[39190, 43961, 47766, 21903, 18599, 1005, 1766..."
3,7,"[40852, 37602, 21137, 32177, 13198, 31683, 121..."
4,13,"[27086, 43086, 14197, 34382, 0, 4210, 27435, 2..."
...,...,...
99995,206202,"[24852, 16797, 49683, 432, 18926, 12919, 0, 22..."
99996,206206,"[27086, 0, 11520, 0, 0, 0, 0, 0, 0, 13045, 0]"
99997,206207,"[25133, 0, 33754, 44632, 13176, 33787, 0, 1618..."
99998,206208,"[43961, 21137, 43086, 0, 22935, 13176, 24964, ..."


In [70]:
#функция предназначена для удаления нулевых элементов из списка,
#а также для формирования срезов из 10-ти элементов
def fetch_ten(l):
  for i in l:
    if i == 0:
      l.remove(i)
  if len(l)>10:
    return l[:10]
  else:
    return l

In [73]:
#применяем функцию "fetch_ten" к столбцу 'product_id'
rec['product_id'] = rec['product_id'].apply(fetch_ten)

In [74]:
#применяем функцию "clean_prediction" к столбцу 'product_id'
rec['product_id'] = rec.apply(clean_prediction, axis=1)

In [75]:
rec

,user_id,product_id
0,1,196 12427 10258 25133 46149 35951 38928 ...
1,2,7963 20785 22825 24768 27413 47209 18523...
2,3,39190 43961 47766 21903 18599 1005 17668...
3,7,40852 37602 21137 32177 13198 31683 1219...
4,13,27086 43086 14197 34382 4210 27435 25134...
...,...,...
99995,206202,24852 16797 49683 432 18926 12919 22220 ...
99996,206206,27086 11520 0 0 13045 0
99997,206207,25133 33754 44632 13176 33787 16185 3397...
99998,206208,43961 21137 43086 22935 13176 24964 2357...


In [76]:
#формируем файл с рекомендациями
submission_6 = rec.to_csv('submission_6.csv', index=False)